In [2]:
import pandas as pd

dataset = pd.read_csv("./train.csv")

dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
dataset  = dataset.dropna()
# independent features
x = dataset.drop('label',axis=1)
# dependent features
y = dataset["label"]
print(x.shape)
print(y.shape)

(18285, 4)
(18285,)


In [4]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [5]:
voc_size = 5000
msg = x.copy()
msg.reset_index(inplace=True)

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\ritik
[nltk_data]     pandey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus =[]
for i in range(0,len(msg)):
    rev = re.sub('[^a-zA-Z]',' ',msg['title'][i])
    rev = rev.lower()
    rev = rev.split()
    rev  = [ps.stem(word) for word in rev if not word in stopwords.words('english')]
    rev = ' '.join(rev)
    corpus.append(rev)


In [14]:
sen_len = []
for sen in corpus:
    sen_len.append(len(sen.split(' ')))
max_len = max(sen_len)
oneHot = [one_hot(words,voc_size)for words in corpus]
embed_doc = pad_sequences(oneHot,padding='pre',maxlen=max_len)
embed_doc[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3830, 2353, 3246, 3035, 2012, 4099, 2181,
       2242, 2003, 3379])

In [22]:
# model
embed_features = 40
model = Sequential()
model.add(Embedding(voc_size,embed_features,input_length=max_len))
model.add(LSTM(100))
model.add(Dense(1,activation='relu'))
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 47, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [23]:
import numpy as np
X = np.array(embed_doc)
y = np.array(y)

from sklearn.model_selection import train_test_split
x_tr, x_te , y_tr, y_te = train_test_split(X,y,test_size=0.33, random_state=42)
 

In [24]:
model.fit(x_tr,y_tr, validation_data=(x_te,y_te),epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 25ms/step - loss: 0.4265 - accuracy: 0.7979 - val_loss: 0.3449 - val_accuracy: 0.9065
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.2684 - accuracy: 0.8650 - val_loss: 0.3033 - val_accuracy: 0.9072
Epoch 3/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1544 - accuracy: 0.9406 - val_loss: 0.3211 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1127 - accuracy: 0.9623 - val_loss: 0.4170 - val_accuracy: 0.9173
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0852 - accuracy: 0.9777 - val_loss: 0.5691 - val_accuracy: 0.9180
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0640 - accuracy: 0.9864 - val_loss: 0.5641 - val_accuracy: 0.9004
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0542 - accuracy: 0.9921 - val_loss: 0.7043 - val_accuracy: 0.9072